In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pathlib
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.1)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

def GetSingleImageFeatureNp(image_features):
    image = tf.io.decode_png(image_features['image/encoded'][0], channels=3)
    xmins = tf.sparse.to_dense(image_features['image/object/bbox/xmin'])[0:1]
    xmaxs = tf.sparse.to_dense(image_features['image/object/bbox/xmax'])[0:1]
    ymins = tf.sparse.to_dense(image_features['image/object/bbox/ymin'])[0:1]
    ymaxs = tf.sparse.to_dense(image_features['image/object/bbox/ymax'])[0:1]
    bbox = tf.concat([ymins, xmins, ymaxs, xmaxs], 0)        
    bbox = tf.transpose(bbox)
    labels_np = tf.sparse.to_dense(image_features['image/object/class/label'])[0].numpy()
    zero_args = np.argwhere(labels_np == 0)
    if zero_args.size > 0:
        num_bbox = zero_args[0][0]
        labels_np = labels_np[:num_bbox]
        bbox = bbox[:num_bbox] 
    return image.numpy(), bbox.numpy(), labels_np

NUM_CLASSES = 8

category_index = {1: {'id': 1, 'name': 'bone'},
                  2: {'id': 2, 'name': 'abdomen'},
                  3: {'id': 3, 'name': 'mediastinum'},
                  4: {'id': 4, 'name': 'liver'},
                  5: {'id': 5, 'name': 'lung'},
                  6: {'id': 6, 'name': 'kidney'},
                  7: {'id': 7, 'name': 'soft_tissue'},
                  8: {'id': 8, 'name': 'pelvis'}}

def _parse_function(example_proto):
    feature_description = {
        'image/height': tf.io.FixedLenFeature((), tf.int64, default_value=1),
        'image/width': tf.io.FixedLenFeature((), tf.int64, default_value=1),
        'image/win_min': tf.io.FixedLenFeature((), tf.float32, default_value=-1024.),
        'image/win_max': tf.io.FixedLenFeature((), tf.float32, default_value=3071.), 
        'image/filename': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/source_id': tf.io.FixedLenFeature((), tf.string, default_value=''),    
        'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/format': tf.io.FixedLenFeature((), tf.string, default_value='jpeg'),
        # Object boxes and classes.
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }  
    return tf.io.parse_single_example(example_proto, feature_description)

In [ ]:
tfr_path = ['/content/drive/MyDrive/data2_big_lesion/tfr_deeplesion.record-00001-of-00030']
batch_size = 1
dataset = tf.data.TFRecordDataset(tfr_path)
dataset = dataset.map(_parse_function)
dataset = dataset.batch(batch_size) 

num_demo = 5
n = 0
train_images_np = []
gt_boxes = []
gt_labels = []
data_iter = iter(dataset)
for i in range(5):
    image_features = next(data_iter)
    image, boxes, labels = GetSingleImageFeatureNp(image_features)
    train_images_np.append(image)
    gt_boxes.append(boxes)
    gt_labels.append(labels)  

#for image_features in dataset:
#  n+=1
#  image, boxes, labels = GetSingleImageFeatureNp(image_features)
#  train_images_np.append(image)
#  gt_boxes.append(boxes)
#  gt_labels.append(labels)
#  if n == 5:
#    break

dummy_scores = np.ones(shape=[1000], dtype=np.float32)  # give boxes a score of 100%

for idx in range(5):
    plt.figure(figsize=(30, 15))
    plt.subplot(1, 5, idx+1)
    plot_detections(train_images_np[idx], gt_boxes[idx], gt_labels[idx], dummy_scores, category_index)
    plt.show()